# Kepler GL

In [ ]:
!pip install keplergl

In [ ]:
import pandas as pd
import geopandas as gpd
from keplergl import KeplerGl

In [ ]:
#Create a basemap
m = KeplerGl(height=600, width=800)

#show the map
m

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
# Create a Dataframe
df = pd.DataFrame(
    {
        "City": ["Shanghai", "Sao paulo", "Cairo", "London", "Toronto", "Sydney"],
        "Country": ["China", "Brazil", "Egypt", "England", "Canada", "Australia"],
        "Latitude": [31.045556, -23.473293, 30.05, 51.514125, 43.666667, -33.861481],
        "Longitude": [121.399722, -46.665803, 31.25, -0.093689, -79.416667, 151.205475],
        "Population": [14608512, 10021437, 7734602, 7421228, 4612187, 4394585],
    }
)


df

In [ ]:
# from google.colab import sheets
# sheet = sheets.InteractiveSheet(df=df)

In [ ]:
# # @title Population

# from matplotlib import pyplot as plt
# df['Population'].plot(kind='hist', bins=20, title='Population')
# plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
m = KeplerGl(height=400, width=500)
m.add_data(data=df, name='cities')

In [ ]:
m

# Adding Geographic Data

In [ ]:
parking_df = pd.read_csv("/content/parking_violations_2015.csv")
parking_df.head()

In [ ]:
parking_df.shape

In [ ]:
parking_df.dropna(subset=["lat", "lon"], how='all', inplace=True)

In [ ]:
parking_df.shape

In [ ]:
gdf = gpd.GeoDataFrame(parking_df, geometry=gpd.points_from_xy(parking_df.lon, parking_df.lat), crs="EPSG:4326")
gdf.head()

In [ ]:
gdf.to_file("/content/parking_2015.geojson",  driver='GeoJSON')


In [ ]:
m = KeplerGl(height=900)
m.add_data(data=gdf, name='parking_points')
m

# 3D Hexagon

In [ ]:
gdf = gpd.read_file("/content/parking_2015.geojson")
gdf.head()

In [ ]:
m = KeplerGl(height=900)
m.add_data(data=gdf, name='3dHexagon')
m

# Animation

In [ ]:
gdf = gpd.read_file("/content/parking_2015.geojson", parse_dates=True)
gdf.head()

In [ ]:
gdf["issue_datetime"] = pd.to_datetime(gdf["issue_datetime"], dayfirst=True)

In [ ]:
gdf.sample(10)

In [ ]:
m = KeplerGl(height=900)
m.add_data(data=gdf, name='animation')
m

# Choropleth Map

In [ ]:
neighborhoods = gpd.read_file("/content/neighbrhoods.geojson")
neighborhoods.head()

In [ ]:
neighborhoods.crs

In [ ]:
neighborhoods = neighborhoods.to_crs("EPSG:4326")

In [ ]:
m = KeplerGl(height=900)
m.add_data(data=neighborhoods, name='choropleth_map')
m

In [ ]:
gdf = gpd.read_file("/content/parking_2015.geojson")
gdf.head()

In [ ]:
neighborhoods = gpd.read_file("/content/neighbrhoods.geojson")
neighborhoods.head()

In [ ]:
gdf.crs == neighborhoods.crs

In [ ]:
neighborhoods = neighborhoods.to_crs("EPSG:4326")

In [ ]:
gdf.crs == neighborhoods.crs

In [ ]:
location_join = gpd.sjoin(gdf, neighborhoods, op="within")

In [ ]:
location_join.head()

In [ ]:
def count_numpoints(joined, polygons, merge_on):
    grouped = joined.groupby(merge_on).size()
    df = grouped.to_frame().reset_index()
    df.columns = [merge_on, "Count"]
    return polygons.merge(df, on=merge_on, how="outer")

numpoints_per_neigh = count_numpoints(location_join, neighborhoods, "NAME")
numpoints_per_neigh.head()

In [ ]:
m = KeplerGl(height=900)
m.add_data(data=numpoints_per_neigh, name='choropleth_map')
m